# Diffusjonsvektet MRI: et bilde av vannmolekylers virrevandring

Her må det stå en INTRODUKSJON (motivasjon for oppgavene)

## Oppgave 1: Frie virrevandrere i en og to dimensjoner

#### 1a)

For å beskrive diffusjon ved hjelp av matematikk, er det naturlig å sette opp en differensialligning som beskriver endringer i vevstetthet som funksjon av posisjon og tid. La $\phi (x,t)$ være tettheten ved posisjon $x$ og tiden $t$. I tillegg innføres en diffusjonskonstant $D$ som en indikasjon på hvor stor diffusjonen er. Sammen gir dette følgende differensialligning:

$$
\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi}{\partial x^2} 
$$


Som mange andre naturlige fenomener, kan vevstettheten $\phi$ beskrives som en normalfordeling med forventningsverdi $\mu = 0$ og varians $\sigma^2 = at$, slik at

$$
\phi(x, t) = \frac{1}{\sqrt{2\pi at}} e^{-\frac{x^2}{2at}}
$$


Videre settes dette inn i ligningen over, for å komme frem til en passende a. 

Vi betrakter venstresiden av ligningen først, og bruker produktregel og deretter kjerneregel for derivasjon. 

$$
\frac{\partial \phi(x,t)}{\partial t} = \frac{\partial}{\partial t} \left(\frac{1}{\sqrt{2\pi at}} e^{-\frac{x^2}{2at}}\right)
$$


$$
= \left(\frac{\partial}{\partial t} \frac{1}{\sqrt{2\pi at}}\right) e^{-\frac{x^2}{2at}} + \frac{1}{\sqrt{2\pi at}} \left(\frac{\partial}{\partial t} e^{-\frac{x^2}{2at}}\right)
$$

$$
= -\frac{1}{2\sqrt{2\pi a} t^{\frac{3}{2}}} e^{-\frac{x^2}{2at}} + \frac{1}{\sqrt{2\pi at}} \frac{x^2}{2at^2} e^{-\frac{x^2}{2at}}
$$

Etter litt opprydning blir dette

$$
\frac{\partial \phi(x,t)}{\partial t} = \frac{1}{2t \sqrt{2\pi at}} e^{-\frac{x^2}{2at}} \left( \frac{x^2}{at} - 1 \right).
$$




Når det gjelder ligningens høyreside, brukes både produktregel og kjerneregel, men ikke før den andre derivajonen. Vi gjør det steg for steg, og deriverer først en gang.

$$
 D \frac{\partial^2 \phi}{\partial x^2} = D\frac{\partial^2}{\partial x^2} \left(\frac{1}{\sqrt{2\pi at}} e^{-\frac{x^2}{2at}}\right)
$$
 
$$
 = -D \frac{1}{\sqrt{2\pi at}} \frac{\partial}{\partial x} \left( \frac{x}{at} e^{-\frac{x^2}{2at}}\right)
$$
 
$$
 = -D \frac{1}{at \sqrt{2\pi at}} \left( \left( \frac{\partial}{\partial x} x \right) e^{-\frac{x^2}{2at}} + x \left(\frac{\partial}{\partial x} e^{-\frac{x^2}{2at}} \right)\right)
$$

$$
 = -D \frac{1}{at \sqrt{2\pi at}} \left( e^{-\frac{x^2}{2at}} - x \frac{x}{at} e^{-\frac{x^2}{2at}} \right)
$$

Skrevet på faktorisert form får vi

$$
= -D \frac{1}{at \sqrt{2\pi at}} e^{-\frac{x^2}{2at}} \left( 1 - \frac{x^2}{at} \right).
$$

Nå settes venstre- og høyreside lik hverandre. Faktorer som er felles strykes, og ligningen løses for a.

$$
\frac{1}{2t \sqrt{2\pi at}} e^{-\frac{x^2}{2at}} \left( \frac{x^2}{at} - 1 \right) = -D \frac{1}{at \sqrt{2\pi at}} e^{-\frac{x^2}{2at}} \left( 1 - \frac{x^2}{at} \right)
$$

$$
\frac{1}{2} = \frac{D}{a}
$$

Vi ender opp med
$$
a = 2D.
$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import ndimage
from numba import njit

In [ ]:
def D(dx=1., dt=1.):
    """
    Gives the constant in Eq. 5.
    """
    return dx ** 2 / (2 * dt)

In [ ]:
def brownian_single_1D(M, pr=0.5):
    """
    Brownian motion for a single particle with M steps.

    Parameters
    ----------
    M : int
        Number of moves.
    pr : float
        Probability for taking a step to the right.

    Returns
    -------
    np.array
        Time array, 1D.
    np.array
        Position array, 1D.
    """
    assert 0. < pr < 1., 'Invalid probability pr'

    positions = np.zeros(M+1)
    random_values = np.random.uniform(0, 1, M)

    for i, value in enumerate(random_values):
        if value <= pr:
            positions[i+1] = positions[i] + 1.
        else:
            positions[i+1] = positions[i] - 1.

    return np.arange(M+1), positions

In [ ]:
def brownian_N_1D(N, M, pr=0.5):
    """
    Brownian motion for N particles with M steps.

    Parameters
    ----------
    N : int
        Number of particles
    M : int
        Number of moves.
    pr : float
        Probability for taking a step to the right.

    Returns
    -------
    np.array
        Time array, 1D.
    np.array
        Position array, 2D.
    """
    assert 0. < pr < 1., 'Invalid probability pr'

    positions = np.zeros((M, N))
    random_values = np.random.uniform(0, 1, (M, N))
    for i in range(M):
        for j in range(N):
            if random_values[i,j] <= pr:
                positions[i,j] = positions[i-1,j] + 1.
            else:
                positions[i,j] = positions[i-1,j] - 1.

    return np.arange(M), positions

In [ ]:
def brownian_N_1D_vectorized(N, M, pr=0.5):
    """
    Brownian motion for N particles with M steps.

    Parameters
    ----------
    N : int
        Number of particles
    M : int
        Number of moves.
    pr : float
        Probability for taking a step to the right.

    Returns
    -------
    np.array
        Time array, 1D.
    np.array
        Position array, 2D. (M, N)
    """
    assert 0. < pr < 1., 'Invalid probability pr'

    random_values = np.random.uniform(0, 1, (M, N))
    steps = np.where(random_values <= pr, +1, -1)
    positions = np.cumsum(steps, axis=0)

    return np.arange(M), positions

In [ ]:
def brownian_N_2D(N, M, pr=0.5, py=0.5):
    """
    Brownian motion for N particles with M steps in two dimensions.

    Parameters
    ----------
    N : int
        Number of particles
    M : int
        Number of moves.
    pr : float
        Probability for taking a step to the right.
    py : float
        Probability for taking a step upwards.

    Returns
    -------
    np.array
        Time array, 1D.
    np.array
        Position array, 3D. Dimension ( M, N, (x,y) ).
    """
    assert 0. < pr < 1., 'Invalid probability pr'
    assert 0. < py < 1., 'Invalid probability py'

    positions = np.zeros((M, N, 2))
    random_values = np.random.uniform(0, 1, size=positions.shape)
    move_horizontal = np.random.uniform(0, 1, size=(M,N)) <= 0.5
    steps_x = np.where(random_values[:,:,0] < pr, +1, -1) * move_horizontal
    steps_y = np.where(random_values[:,:,1] < py, +1, -1) * (1. - move_horizontal)
    positions[:,:,0] += steps_x
    positions[:,:,1] += steps_y

    return np.arange(M), np.cumsum(positions, axis=0)

In [ ]:
@njit
def count_zeros_N_1D(positions: np.ndarray):
    """
    Calculates n(t) for N particles in 1D.

    Parameters
    ----------
    positions : np.ndarray, (M, N)
        Positional array with time as first axis.

    Returns
    -------
    np.ndarray, (M,)
        Time array.
    np.ndarray, (M,)
        Array with n(t) values.
    """
    M, N = positions.shape
    n = np.zeros(M)
    been_to_origo_indicies = []

    for t in range(1, M):
        count = 0
        for i in range(N):
            if positions[t,i] == 0:
                if not (i in been_to_origo_indicies):
                    been_to_origo_indicies.append(i)
                    count += 1

        n[t] = n[t-1] + count / N

    return np.arange(M), n

In [ ]:
@njit
def count_zeros_N_2D(positions: np.ndarray):
    """
    Calculates n(t) for N particles in 2D.

    Parameters
    ----------
    positions : np.ndarray, (M, N, (x,y))
        Positional array with time as first axis.

    Returns
    -------
    np.ndarray, (M,)
        Time array.
    np.ndarray, (M,)
        Array with n(t) values.
    """
    M, N, _ = positions.shape
    n = np.zeros(M)
    been_to_origo_indicies = []

    for t in range(1, M):
        count = 0
        for i in range(N):
            in_origo = positions[t,i,0] == 0 and positions[t,i,1] == 0
            if in_origo:
                if not (i in been_to_origo_indicies):
                    been_to_origo_indicies.append(i)
                    count += 1

        n[t] = n[t-1] + count / N

    return np.arange(M), n

In [ ]:
# Oppgave 1c
M = 10_000

for pr in (0.45, 0.5, 0.55):
    plt.plot(*brownian_single_1D(M, pr), label=r'$p_r$ = '+f'{pr:.2f}')

plt.legend()
plt.xlabel('Step number')
plt.ylabel('Position')
plt.title(f'Brownian motion in 1D - {M = }')
plt.show()

# We observe that the movement favors the right direction if pr > 0.5, as expected.
# M = 10 000 seems high enough to get "random" motion.

In [ ]:
# Testing implementation in Task 1d.
plt.plot(*brownian_N_1D(N=100, M=1000, pr=0.5))
plt.title('Task 1d')
plt.show()

In [ ]:
# Oppgave 1e
# Testing implementation in Task 1e.
# TODO ad timing
plt.plot(*brownian_N_1D_vectorized(N=100, M=1000, pr=0.5))
plt.title('Task 1e')
plt.show()

# We can assume the function is faster since it does not use for-loops. Using only numpy arrays, the machine can
# perform the calculations in C/C++ which makes them go faster than in native Python.


In [ ]:
# Oppgave 1f
t, pos = brownian_N_1D_vectorized(N=1000, M=1000, pr=0.5)
sigma_sq = np.var(pos, axis=1, ddof=1)

a, b = curve_fit(lambda x, a, b: a*x + b, xdata=t, ydata=sigma_sq)[0]
print(f'Linear fit: a = {a:.3e}, b = {b:.3e}')

plt.plot(t, a*t+b, label='Linear regression')
plt.plot(t, sigma_sq, label='Empirical')
plt.title(r'Empirical variance, $\Delta x = 1 = \Delta t$')
plt.xlabel('Time steps')
plt.ylabel(r'Variance $\sigma^2$')
plt.legend()
plt.show()

# Gives a straight line, as one would excpect from 1a.
# Increasing M will only give us more values for sigma and therefore simply lengthen the range og the plot,
# while increasing N will yield a larger number of datapoints for each sigma, such that the variance will
# converge to the analytical variance.

In [ ]:
# Oppgave 1g
N = 4
M = 1000
# Isotrop system
t, pos = brownian_N_2D(N, M)

# For every atom, plot the scatter plot of position in x and y for all time points
for n in range(N):
    plt.plot(pos[:,n,0], pos[:,n,1])

plt.xlabel('x position')
plt.ylabel('y position')
plt.title(f'Brownian motion in 2D \nN = 4, M = 1000, ' + r'$p_r = 0.5 = p_u$')
plt.show()

# Non-isotrop system
t, pos = brownian_N_2D(N, M, pr=0.65, py=0.35)

# For every atom, plot the scatter plot of position in x and y for all time points
for n in range(N):
    plt.scatter(pos[:,n,0], pos[:,n,1])

plt.xlabel('x position')
plt.ylabel('y position')
plt.title(f'Brownian motion in 2D \nN = 4, M = 1000, ' + r'$p_r = 0.65, p_u = 0.35$')
plt.show()

In [ ]:
# Oppgave 1i
N = 100
M = 100_000

# Isotrop system
_, pos = brownian_N_1D_vectorized(N, M)
t, n = count_zeros_N_1D(pos)
plt.plot(t, n, label='1D')

# Non-isotrop system
_, pos = brownian_N_2D(N, M)
t, n = count_zeros_N_2D(pos)
plt.plot(t, n, label='2D')

plt.xlabel('Time steps')
plt.ylabel('n(t)')
plt.title(f'{N = }, {M = }')
plt.axhline(y=1, ls='--', c='k', alpha=0.5)
plt.show()

# 1D Looks correct, converges quite fast
# 2D also looks correct, but converges a lot slower
# Ran simulations with higher N and M for 2D but run time was waaaay too long
# Higher N does not affect convergence, only M does.
# Therefore n(t, M) is a good approximation of P(x=0, t->inf) for high M.
# Convergence also changes quite a lot from run to run, suggesting that M=100_000 is too low for 2D.

# Oppgave 2

In [ ]:
# Def konstanter
DX_MM = 0.004
DT_S = 0.01

## Oppgave 2b)

Ta med tegning(?)

In [ ]:
@njit
def dx_effective(x, y, tumor_pos, area, tk):
    """
    Calculate the effective dx value for points in the xy-plane.

    Parameters
    ----------
    x : foat
        Position in x direction
    y : float
        Position in y direction
    tumor_pos : np.ndarray
        ( m, (x,y) ) array with positions of centers of tumors.
    area : float
        Area of tumors. Same length unit as position.
    tk : np.ndarray
        (m,) array with tumor coefficients.

    Returns
    -------
    np.ndarray
        (x, y) array with effective dx value
    """
    m = tumor_pos.shape[0]
    tk_eff = 1.
    radius = np.sqrt(area / np.pi)

    for mi in range(m):
        tumor = tumor_pos[mi]
        x_dist = tumor[0] - x
        y_dist = tumor[1] - y
        pos = np.asarray((x_dist,y_dist))
        dist = np.linalg.norm(pos)
        if dist < radius:
            tk_eff *= tk[mi]

    return DX_MM * np.sqrt(tk_eff)

## Oppgave 2c og 2d
Her modifiserer vi virrevandrer-funksjonen fra Oppgave 1 for å ta inn en variabel $\Delta x$.

Periodiske randbetingelser er implementert, men kan skrus av og på.

In [ ]:
@njit
def brownian_N_2D_tumor(N, M, tumor_pos, tk, area, L=None, pr=0.5, pu=0.5):
    """
    Brownian motion for N particles with M steps in two dimensions.

    Parameters
    ----------
    N : int
        Number of particles
    M : int
        Number of moves.
    tumor_pos : np.ndarray
        ( m, (x,y) ) array with coordinates of tumor centers.
    tk : np.ndarray
        (m,) array with tumor coefficients.
    area : float
        Area of tumors (constant).
    L : float
        Length of bounding box (0, L) x (0, L).
    pr : float
        Probability for taking a step to the right.
    pu : float
        Probability for taking a step upwards.

    Returns
    -------
    np.array
        Time array, 1D.
    np.array
        Position array, 3D. Dimension ( M, N, (x,y) ).
    """
    assert 0. < pr < 1., 'Invalid probability pr'
    assert 0. < pu < 1., 'Invalid probability pu'
    if L is not None:
        assert L > 0., 'Invalid value L'

    positions = np.zeros((M, N, 2))
    random_values = np.random.uniform(0, 1, size=(M,N))
    move_horizontal = np.random.uniform(0, 1, M) <= 0.5

    for t in range(0, M-1):
        for n in range(N):
            x, y = positions[t,n,0], positions[t,n,1]
            dx_eff: float = dx_effective(x, y, tumor_pos, area, tk)

            if move_horizontal[t]:
                # Keep y coordinate
                positions[t+1,n,1] = positions[t,n,1]

                # Update x coordinate
                if random_values[t,n] <= pr:
                    positions[t+1,n,0] = positions[t,n,0] + dx_eff
                else:
                    positions[t+1,n,0] = positions[t,n,0] - dx_eff
            else:
                # Keep x coordinate
                positions[t+1,n,0] = positions[t,n,0]

                #Update y coordinate
                if random_values[t,n] <= pu:
                    positions[t+1,n,1] = positions[t,n,1] + dx_eff
                else:
                    positions[t+1,n,1] = positions[t,n,1] - dx_eff

            # Periodic boundary conditions
            if L is not None:
                positions[t+1,n] -= np.floor(positions[t+1,n] / L) * L

    return np.arange(M), positions

In [ ]:
# Valus given in task
N = 2
M = 1000
m = 15
L = 0.02  # [mm]

tumor_pos = np.random.uniform(0, L, size=(m,2))
tk = np.full(shape=m, fill_value=0.1)
area = np.pi * DX_MM ** 2
t, pos = brownian_N_2D_tumor(N, M, tumor_pos, tk, area)

# To plot tumors in colors / dx_eff
xmin, xmax = -DX_MM, L + DX_MM
ymin, ymax = -DX_MM, L + DX_MM

# Calculating DX for color plot
n = 200
x = np.linspace(xmin, xmax, n)
y = np.linspace(ymin, ymax, n)
dx = np.zeros((n,n))
for xi in range(n):
    for yi in range(n):
        dx[yi,xi] = dx_effective(x[xi], y[yi], tumor_pos, area, tk)

c = plt.pcolormesh(x, y, dx, cmap='plasma_r')
plt.colorbar(c)

# Plotting random walk
for n in range(N):
    plt.plot(pos[:,n,0], pos[:,n,1])

plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.title('Task 2c')
plt.show()

## Oppgave 2e

In [ ]:
def intensity(nx, ny, xmin, xmax, ymin, ymax, positions):
    """
    Denne funksjonen beregner intensiteten av virrevandrere til hver rute i et rutenett nx*ny.

    Parametere
    ----------
    nx : int 
        Antall rader
    ny : int
        Antall kolonner
    xmin : float
        Minste x-verdi i koordinatsystemet vårt
    xmax : float
        Største x-verdi i koordinatsystemet vårt
    ymin : float
        Minste y-verdi i koordinatsystemet vårt
    ymax :
        Største y-verdi i koordinatsystemet vårt
    positions : np.ndarray[float]
        shape = (M, N, 2). Matrise over posisjonene til N virrevandrere over M tidssteg

    Returnerer
    ----------
    I : np.ndarry[float]
        Normalisert matrise som inneholder intensiteten av virrevandre-bevegelse i hver rute nxny
    """
    I = np.zeros((nx,ny), dtype=float) #Her skal intensitetene lagres
    M = positions.shape[0] #Henter ut tallet M som er første indeks i positions sin shape
    N = positions.shape[1] #Henter ut tallet N som er andre indeks i positions sin shape

    for t in range(M):
        I += np.histogram2d(x = positions[t, :, 0], y = positions[t, :, 1], bins = (nx, ny), range = [[xmin,xmax], [ymin,ymax]])[0] 
        #np.histogram returnerer flere ting, vi ønsker bare første element ([0]) som er selve histogrammet
    return I / (M * N)

## Oppgave 2f

In [ ]:
N = 1000
M = 1000
m = 15
L = 0.02  # [mm]
nx = 40
ny = 40

xmin, xmax = 0, L
ymin, ymax = 0, L

tumor_pos = np.random.uniform(xmin, xmax, size=(m,2))
tk = np.random.uniform(0.3, 0.45, size=m)
area = np.pi * DX_MM ** 2
t, positions = brownian_N_2D_tumor(N, M, tumor_pos, tk, area, L)
I = intensity(nx, ny, xmin, xmax, ymin, ymax, positions)

# Calculating DX for color plot
nr = 200
x = np.linspace(xmin, xmax, nr)
y = np.linspace(ymin, ymax, nr)
dx = np.zeros((nr,nr))
for xi in range(nr):
    for yi in range(nr):
        dx[yi,xi] = dx_effective(x[xi], y[yi], tumor_pos, area, tk)

fig, ax = plt.subplots(
    ncols=2, sharex=False, sharey=False, figsize=(9, 4),
    subplot_kw={'xticks': [], 'yticks': []}  # Removes axes ticks and numbers
)

# Plotting figure of tumors
im = ax[0].pcolormesh(x, y, dx, cmap='PuBuGn_r')
# im = ax[0].imshow(dx, cmap='plasma_r')
ax[0].set(title='Tumorer', xlim=(xmin, xmax), ylim=(ymin,ymax), aspect='equal')
# plt.colorbar(im, ax=ax[0], fraction=0.045)

# Plotting figure of intensity
im = ax[1].imshow(I*100, cmap='PuBuGn', interpolation='bessel')
ax[1].set(title='Intensitet [%]', aspect='equal')
plt.colorbar(im, ax=ax[1], fraction=0.045)

plt.tight_layout()
plt.suptitle('Oppgave 2e')
plt.show()

Drøfting av oppgave 2f.

## Oppgave 2g
### Implementering av Sobel-filter
Vi definerer to funksjoner, en som utfører filtreringen, og en som plotter resultatene.

In [ ]:
def sobel(im):
    """
    Applies Sobel filter on greyscale image.

    Parameters
    ----------
    im : np.ndarray[int]
        (nx, ny) Array containing values for greyscale image.

    Returns
    -------
    Sx : np.ndarray[int]
        (nx, ny) Sobel filter result in X direction.
    Sy : np.ndarray[int]
        (nx, ny) Sobel filter result in Y direction.
    mag : np.ndarray[int]
        (nx, ny) Sobel filter normalized magnitude.
    """
    Sx = ndimage.sobel(im, axis=0)
    Sy = ndimage.sobel(im, axis=1)
    magnitude = np.sqrt(Sx ** 2 + Sy ** 2)
    magnitude *= 255. / np.max(magnitude)  # Normalizing
    return Sx, Sy, magnitude

def plot_sobel(im, show=False):
    """
    Apply Sobel filter on a greyscale image and plot the result.

    Parameters
    ----------
    im : np.ndarray[int]
        (nx, ny) Array containing values for greysacle image.
    show : bool
        Default False. Triggers plt.show().

    Returns
    -------
    plt.figure()
        Subplot figure.
    """
    fig, ax = plt.subplots(
        2, 2, sharex=True, sharey=True, figsize=(9, 6),
        subplot_kw={'xticks': [], 'yticks': []}  # Removes axes ticks and numbers
    )
    cmap = 'Greys_r'  # Greysacle colormap
    Sx, Sy, mag = sobel(im)

    # Plotting original figure
    ax[0,0].imshow(im, cmap)
    ax[0,0].set(title='Original')

    # Plotting Sobel x direction
    ax[0,1].imshow(Sx, cmap)
    ax[0,1].set(title='X')

    # Plotting Sobel y direction
    ax[1,0].imshow(Sy, cmap)
    ax[1,0].set(title='Y')

    # Plotting Sobel magnitude
    ax[1,1].imshow(mag, cmap)
    ax[1,1].set(title='S')

    plt.suptitle('Sobel filter', size='x-large', weight='bold')
    plt.tight_layout()

    if show:
        plt.show()

    return fig

### Anvendelse av Sobel-filter
Nå anvender vi filteret på intensiteten som ble regnet ut deloppgave 2f.

In [ ]:
plot_sobel(I, show=True)

Drøfting